In [1]:
from bs4 import BeautifulSoup
import requests, lxml, os, re
import pandas as pd
import numpy as np


In [2]:


headers = {
            'User-agent':
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"
            }

phrase= 'nlp'

params = {"q": phrase, "hl": "en"}
html = requests.get('https://scholar.google.com/scholar', headers=headers, params=params).text
soup = BeautifulSoup(html, 'lxml')


In [4]:
soup

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">
<html>
<head><meta content="text/html; charset=utf-8" http-equiv="content-type"/><meta content="initial-scale=1" name="viewport"/><title>https://scholar.google.com/scholar?q=nlp&amp;hl=en</title></head>
<body onload="e=document.getElementById('captcha');if(e){e.focus();} if(solveSimpleChallenge) {solveSimpleChallenge(,);}" style="font-family: arial, sans-serif; background-color: #fff; color: #000; padding:20px; font-size:18px;">
<div style="max-width:400px;">
<hr noshade="" size="1" style="color:#ccc; background-color:#ccc;"/><br/>
<form action="index" id="captcha-form" method="post">
<noscript>
<div style="font-size:13px;">
  In order to continue, please enable javascript on your web browser.
</div>
</noscript>
<script async="" defer="" src="https://www.google.com/recaptcha/api.js"></script>
<script>var submitCallback = function(response) {document.getElementById('captcha-form').submit();};</script>
<div class="g-recaptch

define the terms: 
each term will have its size in the circle
each combination of terms will have a connection between the nodes and the thickness will show the conjunction 

In [ ]:
def scrape(phrases:list) -> dict:

    dict_ = {}

    

    return dict_

In [2]:
def extract_number(results_list:list) -> str:

    result_num = 0
    
    for term in results_list:
        if re.findall('about', term.lower()):
            result_num = re.findall('about (.*) results', term.lower())[0]
            result_num = re.sub(',', '', result_num)
            result_num = int(result_num)

    return result_num



In [3]:
def scrape_number_of_results(phrase:str)->int:

    
    params = {"q": phrase, "hl": "en"}
    html = requests.get('https://scholar.google.com/scholar', headers=headers, params=params).text
    soup = BeautifulSoup(html, 'lxml')

        
    # the dictionary will be collected here
    results_list = []

    # Container where all needed dict_ is located
    for result in soup.select('.gs_ab_mdw'):
        results_list.append(result.text)

    result_num = extract_number(results_list)
    
    return result_num

In [4]:

params = {"q": 'nlp and healthcare', "hl": "en"}
html = requests.get('https://scholar.google.com/scholar', headers=headers, params=params).text
soup = BeautifulSoup(html, 'lxml')

    
# the dictionary will be collected here
results_list = []

# Container where all needed dict_ is located
for result in soup.select('.gs_ab_mdw'):
    print(result.text)
    results_list.append(result.text)


In [5]:
def combine_phrases(phrases:list)->list:
    """
    gets all phrases and creates a list of their combination 
    a list of str(phrase 1 + " AND " + phrase 2)
    """
    list_of_phrase_comb = []
    # combination of terms
    for i in range(len(phrases)):
        for j in range(len(phrases)):
            if i>j:
                new_phrase = phrases[i] + " AND " + phrases[j]
                list_of_phrase_comb.append(new_phrase)

    return list_of_phrase_comb

In [6]:
def determine_node_size(phrases:list) -> dict:
    """
    get the list of phrases
    output a dictionary of their size (number of search results for each phrase)
    """
    sizes_dict = {}
    # single term search for size of node
    for phrase in phrases:
        sizes_dict[phrase] = scrape_number_of_results(phrase)
        
    return sizes_dict

In [7]:
def create_graph(phrases):
    
    graph_ = {'nodes':[], 'links':[]}
    
    sizes_dict =  determine_node_size(phrases)

    
    # for nodes
    # {'nodes' : [{'id': , 'group':2 , 'size':name_freq[char]}]}
    for term, numb_ in sizes_dict.items():
        graph_['nodes'].append({'id': term, 'group': 1, "size":float(np.log(numb_)) })
    
    list_of_phrase_comb = []
    # combination of terms
    for i in range(len(phrases)):
        for j in range(len(phrases)):
            if i>j:
                new_phrase = "\"" + phrases[i] + "\"" + " AND " + "\"" + phrases[j] + "\""
                print(new_phrase)
                results_num = scrape_number_of_results(phrase=new_phrase)

                graph_['links'].append({'source': phrases[i], 
                                        'target': phrases[j],
                                        'value': float(results_num), 
                                        'color': float(results_num)})
    
    
    return graph_

In [8]:
graph_ = create_graph(phrases=['natural language processing', 'decision support systems', 'clinical setting', 'artificial intelligence' , 'healthcare'])

C:\Users\mitra\AppData\Local\Temp\ipykernel_20564\2097357067.py:11: RuntimeWarning: divide by zero encountered in log
  graph_['nodes'].append({'id': term, 'group': 1, "size":float(np.log(numb_)) })


"decision support systems" AND "natural language processing"
"clinical setting" AND "natural language processing"
"clinical setting" AND "decision support systems"
"artificial intelligence" AND "natural language processing"
"artificial intelligence" AND "decision support systems"
"artificial intelligence" AND "clinical setting"
"healthcare" AND "natural language processing"
"healthcare" AND "decision support systems"
"healthcare" AND "clinical setting"
"healthcare" AND "artificial intelligence"


In [9]:
print(graph_)

{'nodes': [{'id': 'natural language processing', 'group': 1, 'size': -inf}, {'id': 'decision support systems', 'group': 1, 'size': -inf}, {'id': 'clinical setting', 'group': 1, 'size': -inf}, {'id': 'artificial intelligence', 'group': 1, 'size': -inf}, {'id': 'healthcare', 'group': 1, 'size': -inf}], 'links': [{'source': 'decision support systems', 'target': 'natural language processing', 'value': 0.0, 'color': 0.0}, {'source': 'clinical setting', 'target': 'natural language processing', 'value': 0.0, 'color': 0.0}, {'source': 'clinical setting', 'target': 'decision support systems', 'value': 0.0, 'color': 0.0}, {'source': 'artificial intelligence', 'target': 'natural language processing', 'value': 0.0, 'color': 0.0}, {'source': 'artificial intelligence', 'target': 'decision support systems', 'value': 0.0, 'color': 0.0}, {'source': 'artificial intelligence', 'target': 'clinical setting', 'value': 0.0, 'color': 0.0}, {'source': 'healthcare', 'target': 'natural language processing', 'val

In [ ]:
i=0

In [56]:

import json
with open(f"C:\\Users\\mitra\\PassivePy\\PassivePyCode\\SENG Scrape\\graph_{i}.json", 'w') as fp:
    json.dump(graph_, fp)
    i+=1

add something that can plot terms versus time

In [ ]:
    def matrix_to_edge(self, cooccurrence_matrix:np.matrix, cooccurrence_matrix_with_senti:np.matrix,
     pop_names_df:pd.DataFrame, top_n_popular_names:list)->Graph:

        graph_ = {'nodes':[], 'links':[]}
        shape = cooccurrence_matrix.shape[0]
        name_freq = pop_names_df.sum()


        # for nodes
        # {'nodes' : [{'id': , 'group':2 , 'size':name_freq[char]}]}
        for char in top_n_popular_names:
            graph_['nodes'].append({'id': char, 'group': 1, "size":np.float(np.log(name_freq)[char])*5 })


        # for edges
        # {'links' : [{'source': , 'target': , 'value': , 'color': }]}
        for i in range(shape):
            for j in range(shape):
                if i>j:
                    graph_['links'].append({'source': top_n_popular_names[i], 
                                            'target': top_n_popular_names[j],
                                        'value': np.float(cooccurrence_matrix[i, j]), 
                                        'color':cooccurrence_matrix_with_senti[i,j]})
                    
        return graph_


In [38]:
for term in results_list:
    if re.findall('about', term.lower()):
        result_num = re.findall('about (.*) results', term.lower())[0]
        result_num = re.sub(',', '', result_num)
        result_num = int(result_num)
        print(result_num)

39200


In [26]:
result_num

[]

In [30]:
re.findall("about", "About 39,200 results (0.03 sec)".lower())

['about']